**Capturar fotos**

In [ ]:
import cv2
import os

def capture_photos(person_name, num_photos=10):
    # Crear la carpeta "Personas" si no existe
    base_dir = 'Nom'
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    
    # Crear la carpeta específica para la persona si no existe
    person_dir = os.path.join(base_dir, person_name)
    if not os.path.exists(person_dir):
        os.makedirs(person_dir)

    cap = cv2.VideoCapture(1)  # Cambiar a 0 o 1 para la cámara adecuada
    count = 0

    while count < num_photos:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("Captura de Fotos", frame)

        # Presionar 'c' para capturar una foto
        if cv2.waitKey(1) & 0xFF == ord('c'):
            # Guardar la imagen
            img_path = os.path.join(person_dir, f"{person_name}_{count}.jpg")
            cv2.imwrite(img_path, frame)
            count += 1
            print(f"Foto {count} guardada en {img_path}")

        # Presionar 'q' para salir antes
        elif cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Ejemplo de uso
capture_photos('Juan', num_photos=10)


Importaciones


In [1]:
import dlib
import cv2
import numpy as np
import os
import json
from imutils import face_utils
from imutils import rotate_bound, translate, adjust_brightness_contrast

# Inicializar el detector de caras de dlib y el predictor de puntos faciales
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("Modelos/shape_predictor_68_face_landmarks.dat")
face_rec_model = dlib.face_recognition_model_v1("Modelos/dlib_face_recognition_resnet_model_v1.dat")

**Modelo**

In [44]:
# Función para obtener los descriptores faciales de todas las caras en una imagen
def get_face_descriptors_from_image(image_path):
    image = cv2.imread(image_path)
    # augmented_images = augment_image(image)

    face_descriptors = []
    # for aug_image in [image] + augmented_images:
    for aug_image in [image]: 
        gray = cv2.cvtColor(aug_image, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 1)

        for face in faces:
            shape = predictor(gray, face)
            face_descriptor = face_rec_model.compute_face_descriptor(aug_image, shape)
            face_descriptors.append(np.array(face_descriptor))

    return face_descriptors

# Obtener los descriptores faciales de las fotos capturadas
def get_face_descriptors(image_folder):
    descriptors = []
    image_names = os.listdir(image_folder)
    for i, img_name in enumerate(image_names):
        img_path = os.path.join(image_folder, img_name)
        descriptor = get_face_descriptors_from_image(img_path)
        if descriptor:
            descriptors.extend(descriptor)
        print(f'Procesado {i+1} de {len(image_names)} imágenes')
    return descriptors

# Guardar los descriptores en un archivo JSON
def save_descriptors(person_name, descriptors):
    # Crear la carpeta "Modelos" si no existe
    model_dir = 'Modelos'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    descriptors_list = [desc.tolist() for desc in descriptors]
    data = {person_name: descriptors_list}
    file_path = os.path.join(model_dir, "face_descriptors.json")
    
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            existing_data = json.load(file)
            existing_data.update(data)
        with open(file_path, "w") as file:
            json.dump(existing_data, file)
    else:
        with open(file_path, "w") as file:
            json.dump(data, file)

# Cargar los descriptores desde un archivo JSON
def load_descriptors():
    file_path = os.path.join('Modelos', "face_descriptors.json")
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            data = json.load(file)
            for person in data:
                data[person] = np.array(data[person])
            return data
    else:
        return {}

# Entrenar el modelo con las fotos capturadas de una nueva persona
def train_new_person(person_name, image_folder):
    descriptors = get_face_descriptors(image_folder)
    if descriptors:
        save_descriptors(person_name, descriptors)
    else:
        print(f"No se encontraron descriptores para {person_name}")

# Función para reconocer personas en una imagen
def recognize_person(image_path, threshold=0.4):
    known_descriptors = load_descriptors()
    test_descriptors = get_face_descriptors_from_image(image_path)

    if not test_descriptors:
        return "No se detectó ninguna cara en la imagen."

    results = set()  # Usar un conjunto para almacenar resultados únicos
    for test_descriptor in test_descriptors:
        min_distance = float("inf")
        recognized_person = None

        for person_name, descriptors in known_descriptors.items():
            distances = np.linalg.norm(descriptors - test_descriptor, axis=1)
            distance = np.min(distances)
            if distance < min_distance:
                min_distance = distance
                recognized_person = person_name

        if min_distance < threshold:
            results.add(f"Reconocido como {recognized_person} con una distancia de {min_distance:.2f}")
        else:
            results.add("Persona no reconocida.")

    return list(results)  # Convertir el conjunto a lista para la salida

**Entrenar**

In [ ]:
# image_folder = os.path.join("lfw\lfw", "George_W_Bush")

person_name = "Lalo"  # Nombre de la carpeta en "Personas" para esta persona
image_folder = os.path.join("Personas", person_name)
train_new_person(person_name, image_folder)

**Buscar Persona**

In [54]:
# Prueba con una imagen nueva
test_image_path = "imgs/f.jpg"
print(recognize_person(test_image_path))

['Persona no reconocida.', 'Reconocido como Franco con una distancia de 0.38', 'Reconocido como Lalo con una distancia de 0.33']


Mostrar lista de personas reconocibles

In [27]:
def list_models():
    known_descriptors = load_descriptors()
    for person in known_descriptors:
        print(person)

list_models()

Jona
Franco
Lalo


**codigo adicional**

In [ ]:
# Función para realizar aumentación de datos en una imagen
# def augment_image(image):
#     augmented_images = []

#     # Rotaciones
#     angles = [-15, -10, -5, 5, 10, 15]
#     for angle in angles:
#         rotated = rotate_bound(image, angle)
#         augmented_images.append(rotated)

#     # Traslaciones
#     shifts = [(-10, 0), (10, 0), (0, -10), (0, 10)]
#     for (x_shift, y_shift) in shifts:
#         translated = translate(image, x_shift, y_shift)
#         augmented_images.append(translated)

#     # Ajustes de brillo y contraste
#     adjustments = [(1.2, 10), (0.8, -10)]
#     for (alpha, beta) in adjustments:
#         adjusted = adjust_brightness_contrast(image, alpha, beta)
#         augmented_images.append(adjusted)

#     return augmented_images

In [26]:
# Eliminar una persona del archivo JSON de descriptores faciales
def delete_person(person_name):
    file_path = os.path.join('Modelos', "face_descriptors.json")
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            data = json.load(file)
        if person_name in data:
            del data[person_name]
            with open(file_path, "w") as file:
                json.dump(data, file)
            print(f"Descriptores de {person_name} eliminados.")
        else:
            print(f"No se encontraron descriptores para {person_name}.")
    else:
        print("El archivo de descriptores no existe.")


delete_person("Personas/Lalo")

Descriptores de Personas/Lalo eliminados.


In [58]:

# Función para obtener los descriptores faciales de todas las caras en una imagen y devolver las coordenadas de las caras
def get_face_descriptors_from_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error al cargar la imagen {image_path}")
        return [], []

    face_descriptors = []
    face_coordinates = []

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 1)

    if len(faces) == 0:
        print(f"No se detectaron caras en la imagen {image_path}")
    else:
        for face in faces:
            shape = predictor(gray, face)
            face_descriptor = face_rec_model.compute_face_descriptor(image, shape)
            face_descriptors.append(np.array(face_descriptor))
            face_coordinates.append((face.left(), face.top(), face.right(), face.bottom()))

    return face_descriptors, face_coordinates

# Obtener los descriptores faciales de las fotos capturadas
def get_face_descriptors(image_folder):
    descriptors = []
    image_names = os.listdir(image_folder)
    for i, img_name in enumerate(image_names):
        img_path = os.path.join(image_folder, img_name)
        descriptor, _ = get_face_descriptors_from_image(img_path)
        if descriptor:
            descriptors.extend(descriptor)
        print(f'Procesado {i+1} de {len(image_names)} imágenes')
    return descriptors

# Guardar los descriptores en un archivo JSON
def save_descriptors(person_name, descriptors):
    # Crear la carpeta "Modelos" si no existe
    model_dir = 'Modelos'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    descriptors_list = [desc.tolist() for desc in descriptors]
    data = {person_name: descriptors_list}
    file_path = os.path.join(model_dir, "face_descriptors.json")
    
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            existing_data = json.load(file)
            existing_data.update(data)
        with open(file_path, "w") as file:
            json.dump(existing_data, file)
    else:
        with open(file_path, "w") as file:
            json.dump(data, file)

# Cargar los descriptores desde un archivo JSON
def load_descriptors():
    file_path = os.path.join('Modelos', "face_descriptors.json")
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            data = json.load(file)
            for person in data:
                data[person] = np.array(data[person])
            return data
    else:
        return {}

# Entrenar el modelo con las fotos capturadas de una nueva persona
def train_new_person(person_name, image_folder):
    descriptors = get_face_descriptors(image_folder)
    if descriptors:
        save_descriptors(person_name, descriptors)
    else:
        print(f"No se encontraron descriptores para {person_name}")

# Función para reconocer personas en una imagen y encuadrar los rostros
def recognize_person(image_path, threshold=0.4):
    known_descriptors = load_descriptors()
    test_descriptors, face_coordinates = get_face_descriptors_from_image(image_path)

    if not test_descriptors:
        return "No se detectó ninguna cara en la imagen."

    results = set()  # Usar un conjunto para almacenar resultados únicos
    image = cv2.imread(image_path)  # Cargar la imagen para dibujar rectángulos

    for i, test_descriptor in enumerate(test_descriptors):
        min_distance = float("inf")
        recognized_person = None

        for person_name, descriptors in known_descriptors.items():
            distances = np.linalg.norm(descriptors - test_descriptor, axis=1)
            distance = np.min(distances)
            if distance < min_distance:
                min_distance = distance
                recognized_person = person_name

        x, y, w, h = face_coordinates[i][0], face_coordinates[i][1], face_coordinates[i][2] - face_coordinates[i][0], face_coordinates[i][3] - face_coordinates[i][1]
        if min_distance < threshold:
            results.add(f"Reconocido como {recognized_person} con una distancia de {min_distance:.2f}")
            # Dibujar un rectángulo alrededor de la cara reconocida
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            # Poner el nombre de la persona reconocida
            cv2.putText(image, recognized_person, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            results.add("Persona no reconocida.")
            # Dibujar un rectángulo alrededor de la cara no reconocida
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Guardar la imagen con las caras encuadradas
    output_path = os.path.join("output", os.path.basename(image_path))
    if not os.path.exists("output"):
        os.makedirs("output")
    cv2.imwrite(output_path, image)

    return list(results)  # Convertir el conjunto a lista para la salida



# Reconocer personas en una imagen específica
image_path = "imgs/f.jpg"
recognition_results = recognize_person(image_path)
print(recognition_results)


['Persona no reconocida.', 'Reconocido como Franco con una distancia de 0.38', 'Reconocido como Lalo con una distancia de 0.33']
